# Installs

In [1]:
%pip install torchvision==0.14.1+cu117 torchtext==0.14.1 torchaudio==0.13.1 torchdata==0.5.1 --extra-index-url https://download.pytorch.org/whl/cu117 -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 2.4.0 requires torch>=2.1.0, but you have torch 1.13.1+cu117 which is incompatible.
Note: you may need to restart the kernel to use updated packages.



This may take a while

In [2]:
#!pip install torchsummaryx==1.1.0
!pip install wandb --quiet
!pip install python-Levenshtein -q
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget -q
!pip install --upgrade setuptools packaging
%cd ctcdecode
!pip install . -q
%cd ..

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 1102 (delta 16), reused 32 (delta 14), pack-reused 1063 (from 1)
Receiving objects: 100% (1102/1102), 782.27 KiB | 12.42 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/kaggle/working/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Counting objects: 100% (26/26), done.        
remote: Compressing objects: 100% (9/9), done.        
remote: Total 82 (delta 19), reused 17 (delta 17), pack-reused 56 (from 1)        
Receiving objects: 100% (82/82), 13.34 KiB | 3.34 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Cloning int

# Imports

In [3]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
#from torchsummaryX import summary
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import torchaudio.transforms as tat

from sklearn.metrics import accuracy_score
import gc

import zipfile
import pandas as pd
from tqdm import tqdm
import os
import datetime

# imports for decoding and distance calculation
import Levenshtein
import ctcdecode
from ctcdecode import CTCBeamDecoder

import warnings
warnings.filterwarnings('ignore')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


# Dataset and Dataloader

In [4]:
# ARPABET PHONEME MAPPING
# DO NOT CHANGE

CMUdict_ARPAbet = {
    "" : " ",
    "[SIL]": "-", "NG": "G", "F" : "f", "M" : "m", "AE": "@",
    "R"    : "r", "UW": "u", "N" : "n", "IY": "i", "AW": "W",
    "V"    : "v", "UH": "U", "OW": "o", "AA": "a", "ER": "R",
    "HH"   : "h", "Z" : "z", "K" : "k", "CH": "C", "W" : "w",
    "EY"   : "e", "ZH": "Z", "T" : "t", "EH": "E", "Y" : "y",
    "AH"   : "A", "B" : "b", "P" : "p", "TH": "T", "DH": "D",
    "AO"   : "c", "G" : "g", "L" : "l", "JH": "j", "OY": "O",
    "SH"   : "S", "D" : "d", "AY": "Y", "S" : "s", "IH": "I",
    "[SOS]": "[SOS]", "[EOS]": "[EOS]"
}

CMUdict = list(CMUdict_ARPAbet.keys())
ARPAbet = list(CMUdict_ARPAbet.values())


PHONEMES = CMUdict[:-2]
LABELS = ARPAbet[:-2]



In [5]:
# You might want to play around with the mapping as a sanity check here



### Train Data

In [6]:
import torchaudio

class PermuteBlock(torch.nn.Module):
    def forward(self, x):
        return x.transpose(1, 2)
    
audio_transforms = nn.Sequential(
    PermuteBlock(), 
    torchaudio.transforms.FrequencyMasking(freq_mask_param=5),
    torchaudio.transforms.TimeMasking(time_mask_param=100),
    PermuteBlock()
)

In [7]:
class AudioDataset(torch.utils.data.Dataset):

    # For this homework, we give you full flexibility to design your data set class.
    # Hint: The data from HW1 is very similar to this HW

    #TODO
    def __init__(self, root, phonemes = PHONEMES,context=0, partition= "train-clean-100",audio_transformation=None): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes
        self.transformation = audio_transformation
        '''
        Initializes the dataset.

        INPUTS: What inputs do you need here?
        '''

        # Load the directory and all files in them

        self.mfcc_dir = os.path.join(root, partition, 'mfcc')
        self.transcript_dir = os.path.join(root, partition, 'transcript')

        self.mfcc_files = sorted(os.listdir(self.mfcc_dir))
        self.transcript_files = sorted(os.listdir(self.transcript_dir))

        self.PHONEMES = PHONEMES
        
         # Making sure that we have the same no. of mfcc and transcripts
        assert len(self.mfcc_files) == len(self.transcript_files)

        #TODO
        # WHAT SHOULD THE LENGTH OF THE DATASET BE?
        self.length = len(self.mfcc_files)

        #TODO
        # HOW CAN WE REPRESENT PHONEMES? CAN WE CREATE A MAPPING FOR THEM?
        # HINT: TENSORS CANNOT STORE NON-NUMERICAL VALUES OR STRINGS

        #TODO
        # CREATE AN ARRAY OF ALL FEATUERS AND LABELS
        # WHAT NORMALIZATION TECHNIQUE DID YOU USE IN HW1? CAN WE USE IT HERE?
        '''
        You may decide to do this in __getitem__ if you wish.
        However, doing this here will make the __init__ function take the load of
        loading the data, and shift it away from training.
        '''
        self.mfccs, self.transcripts = [], []
        
        for i in range(len(self.mfcc_files)):
        #   Load a single mfcc
            mfcc        = np.load(os.path.join(self.mfcc_dir, self.mfcc_files[i]))
        #   Do Cepstral Normalization of mfcc (explained in writeup)
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)
        #   Load the corresponding transcript
            transcript  = np.load(os.path.join(self.transcript_dir, self.transcript_files[i]))
            if transcript[0] == "[SOS]" and transcript[-1] == "[EOS]":
                transcript = transcript[1:-1]
            # (Is there an efficient way to do this without traversing through the transcript?)
            # Note that SOS will always be in the starting and EOS at end, as the name suggests.
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            transcript = np.vectorize(self.phonemes.index)(transcript)
        
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)
            
            

    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length

    def __getitem__(self, ind):
        '''
        TODO: RETURN THE MFCC COEFFICIENTS AND ITS CORRESPONDING LABELS

        If you didn't do the loading and processing of the data in __init__,
        do that here.

        Once done, return a tuple of features and labels.
        '''

        mfcc = torch.FloatTensor(self.mfccs[ind])
        transcript = torch.tensor(self.transcripts[ind])
        return mfcc, transcript


    def collate_fn(self,batch):
        '''
        TODO:
        1.  Extract the features and labels from 'batch'
        2.  We will additionally need to pad both features and labels,
            look at pytorch's docs for pad_sequence
        3.  This is a good place to perform transforms, if you so wish.
            Performing them on batches will speed the process up a bit.
        4.  Return batch of features, labels, lenghts of features,
            and lengths of labels.
        '''                            

        # HINT: CHECK OUT -> pad_sequence (imported above)
        # Also be sure to check the input format (batch_first)
        batch_mfcc, batch_transcript = [], []
        lengths_mfcc, lengths_transcript = [], []
        for (m, t) in batch:
            batch_mfcc.append(m)
            lengths_mfcc.append(len(m))
            batch_transcript.append(t)
            lengths_transcript.append(len(t))
        
       
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first = True)
        
        if self.transformation is not None:
            batch_mfcc_pad = self.transformation(batch_mfcc_pad)
        
        batch_transcript_pad = pad_sequence(batch_transcript, batch_first = True)



        # You may apply some transformation, Time and Frequency masking, here in the collate function;
        # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
        #                  -> Would we apply transformation on the validation set as well?
        #                  -> Is the order of axes / dimensions as expected for the transform functions?

        # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, batch_transcript_pad, torch.tensor(lengths_mfcc), torch.tensor(lengths_transcript)



### Test Data

In [8]:
# Test Dataloader
#TODO
class AudioDatasetTest(torch.utils.data.Dataset):
    def __init__(self, root, phonemes = PHONEMES,context=0, partition= "test-clean"): # Feel free to add more arguments

        self.context    = context
        self.phonemes   = phonemes

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data using root
        self.mfcc_dir = os.path.join(root, partition, 'mfcc')
        # TODO: List files in sefl.mfcc_dir using os.listdir in sorted order
        self.mfcc_files   = sorted(os.listdir(self.mfcc_dir))


        self.length = len(self.mfcc_files)

        self.mfccs, self.transcripts = [], []

        for i in range(len(self.mfcc_files)):
                  #   Load a single mfcc
            mfcc = np.load(os.path.join(self.mfcc_dir, self.mfcc_files[i]))
            mfcc = (mfcc - np.mean(mfcc, axis=0)) / np.std(mfcc, axis=0)

            self.mfccs.append(mfcc)
            
            

    def __len__(self):

        '''
        TODO: What do we return here?
        '''
        return self.length
    
    
    def __getitem__(self, ind):
        mfcc = self.mfccs[ind]
        return torch.FloatTensor(mfcc)
      
    def collate_fn(self, batch):
        batch_mfcc = []
        lengths_mfcc = []
        
        for mfcc in batch:
            batch_mfcc.append(mfcc)   
            lengths_mfcc.append(len(mfcc))
      
        batch_mfcc_pad = pad_sequence(batch_mfcc, batch_first = True)
            

            # You may apply some transformation, Time and Frequency masking, here in the collate function;
            # Food for thought -> Why are we applying the transformation here and not in the __getitem__?
            #                  -> Would we apply transformation on the validation set as well?
            #                  -> Is the order of axes / dimensions as expected for the transform functions?
            
            # Return the following values: padded features, padded labels, actual length of features, actual length of the labels
        return batch_mfcc_pad, torch.tensor(lengths_mfcc)


### Config - Hyperparameters

In [9]:


# Feel free to add more items here
config = {
    "beam_width" : 1,
    "lr"         : 2e-3,
    "epochs"     : 90,
    "batch_size" : 64  # Increase if your device can handle it
}

# You may pass this as a parameter to the dataset class above
# This will help modularize your implementation
transforms = [] # set of tranformations

### Data loaders

In [10]:
# get me RAMMM!!!!
import gc
gc.collect()

0

In [11]:
root="/kaggle/input/11-785-hw3p2-f24/11785-f24-hw3p2"
#TODO: Create a dataset object using the AudioDataset class for the training data
train_data = AudioDataset(root=root,audio_transformation=audio_transforms)

# TODO: Create a dataset object using the AudioDataset class for the validation data
val_data = AudioDataset(root=root, partition="dev-clean")

# TODO: Create a dataset object using the AudioTestDataset class for the test data
test_data = AudioDatasetTest(root=root)

In [12]:
# Do NOT forget to pass in the collate function as parameter while creating the dataloader
train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 8,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True,
    collate_fn = train_data.collate_fn
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False,
    collate_fn = val_data.collate_fn
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 4,
    batch_size  = 1,
    pin_memory  = True,
    shuffle     = False,
    collate_fn = test_data.collate_fn
)

print("Batch size: ", config['batch_size'])
print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Val dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  64
Train dataset samples = 28539, batches = 446
Val dataset samples = 2703, batches = 43
Test dataset samples = 2620, batches = 2620


In [13]:
# sanity check
for data in train_loader:
    x, y, lx, ly = data
    print(x.shape, y.shape, lx.shape, ly.shape)
    break

torch.Size([64, 1671, 28]) torch.Size([64, 200]) torch.Size([64]) torch.Size([64])


# NETWORK

## ASR Network

### Pyramid Bi-LSTM (pBLSTM)

In [14]:
# Utils for network
torch.cuda.empty_cache()

In [15]:
class pBLSTM(torch.nn.Module):

    '''
    Pyramidal BiLSTM
    Read the write up/paper and understand the concepts and then write your implementation here.

    At each step,
    1. Pad your input if it is packed (Unpack it)
    2. Reduce the input length dimension by concatenating feature dimension
        (Tip: Write down the shapes and understand)
        (i) How should  you deal with odd/even length input? 
        (ii) How should you deal with input length array (x_lens) after truncating the input?
    3. Pack your input
    4. Pass it into LSTM layer

    To make our implementation modular, we pass 1 layer at a time.
    '''
    
    def __init__(self, input_size, hidden_size):
        super(pBLSTM, self).__init__()

        self.blstm = nn.LSTM(input_size = 2*input_size, hidden_size = hidden_size, num_layers = 1, bidirectional = True, dropout = 0.2, batch_first = True) 

    def forward(self, x_packed): # x_packed is a PackedSequence
        
        x , lengths = pad_packed_sequence(x_packed, batch_first = True)
        
        x, x_lens = self.trunc_reshape(x, lengths)
        
        x = pack_padded_sequence(x, x_lens, batch_first = True, enforce_sorted= False)
        
        x , h= self.blstm(x)
        
        return x

    def trunc_reshape(self, x, x_lens): 
        
        if x.shape[1]%2 != 0:
            x= x[:,:-1,:]

        x = x.reshape(x.shape[0],x.shape[1]//2, x.shape[2]*2)
        x_lens  = x_lens//2
        return x, x_lens

### Encoder

In [16]:
class LockedDropout(nn.Module):
    def __init__(self, drop_prob):
        super(LockedDropout, self).__init__()
        self.prob = drop_prob
    def forward(self, x):
        if not self.training or not self.prob: # turn it off during inference
            return x
        x, x_lens = pad_packed_sequence(x, batch_first = True)
        m = x.new_empty(x.size(0), 1, x.size(2),requires_grad=False).bernoulli_(1 - self.prob)
        mask = m / (1 - self.prob)
        mask = mask.expand_as(x)
        out = x * mask
        out = pack_padded_sequence(out,x_lens, batch_first = True, enforce_sorted= False)
        return out

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# Define the Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride, padding)
        self.bn2 = nn.BatchNorm1d(out_channels)
        
        # Define a skip connection if input and output channels differ
        self.skip_connection = None
        if in_channels != out_channels:
            self.skip_connection = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1)
        
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.gelu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Add skip connection
        if self.skip_connection is not None:
            identity = self.skip_connection(identity)
        
        out += identity
        return F.gelu(out)

# Modified Encoder class with Residual Blocks in embed
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        
        self.embed = nn.Sequential(
            PermuteBlock(),
            ResidualBlock(input_size, 128),
            ResidualBlock(128, 256),
            PermuteBlock(),
            nn.Dropout(p=0.2)
        )

        self.pBLSTMs = nn.Sequential(
            pBLSTM(input_size=256, hidden_size=hidden_size),
            LockedDropout(0.4),
            pBLSTM(input_size=2*hidden_size, hidden_size=hidden_size),
            LockedDropout(0.3)
        )
         
    def forward(self, x, lens):
        x = self.embed(x)
        lens = lens.clamp(max=x.shape[1]).cpu() 
        x = pack_padded_sequence(x, lens, batch_first=True, enforce_sorted=False)
        x = self.pBLSTMs(x)
        outputs, lens = pad_packed_sequence(x, batch_first=True)
        
        return outputs, lens


### Decoder

In [18]:
class Decoder(torch.nn.Module):

    def __init__(self, embed_size, output_size= 41):
        super().__init__()

        self.mlp = torch.nn.Sequential(
            PermuteBlock(), torch.nn.BatchNorm1d(embed_size), PermuteBlock(),
            #TODO define your MLP arch. Refer HW1P2
            #Use Permute Block before and after BatchNorm1d() to match the size
            nn.Linear(embed_size, 2048),
            nn.GELU(),
            PermuteBlock(), torch.nn.BatchNorm1d(2048), PermuteBlock(),
            nn.Dropout(0.25),
            nn.Linear(2048, 1024),
            nn.GELU(),
            PermuteBlock(), torch.nn.BatchNorm1d(1024), PermuteBlock(),
            nn.Dropout(0.2),
            nn.Linear(1024, 512),
            nn.GELU(),
            PermuteBlock(), torch.nn.BatchNorm1d(512), PermuteBlock(),
            nn.Dropout(0.1),
            nn.Linear(512, output_size)
        )
        
        self.softmax = torch.nn.LogSoftmax(dim=2)

    def forward(self, encoder_out):
        #TODO call your MLP
        #TODO Think what should be the final output of the decoder for the classification 
        out = self.mlp(encoder_out)
        out = self.softmax(out)

        return out 

In [19]:
class ASRModel(torch.nn.Module):

    def __init__(self, input_size, embed_size= 192, output_size= len(PHONEMES)):
        super().__init__()

        # self.augmentations  = torch.nn.Sequential(
        #     #TODO Add Time Masking/ Frequency Masking
        #     #Hint: See how to use PermuteBlock() function defined above
        #     PermuteBlock(), 
        #     torchaudio.transforms.FrequencyMasking(freq_mask_param=10),
        #     torchaudio.transforms.TimeMasking(time_mask_param=100),
        #     PermuteBlock(),
        # ) # did augmentation in the collate_fn
        self.encoder        = Encoder(input_size, embed_size)# TODO: Initialize Encoder
        self.decoder        = Decoder(embed_size*2, output_size) # TODO: Initialize Decoder 

        
    
    def forward(self, x, lengths_x):
        
        # if self.training:
        #     x = self.augmentations(x)

        encoder_out, encoder_lens   = self.encoder(x, lengths_x)
        decoder_out                 = self.decoder(encoder_out)

        return decoder_out, encoder_lens

## Initialize ASR Network

In [20]:
model = ASRModel(
    input_size  = 28,
    embed_size  = 512,
    output_size = len(PHONEMES)
).to(device)
print(model)
#summary(model, x.to(device), lx.to(device))

ASRModel(
  (encoder): Encoder(
    (embed): Sequential(
      (0): PermuteBlock()
      (1): ResidualBlock(
        (conv1): Conv1d(28, 128, kernel_size=(3,), stride=(1,), padding=(1,))
        (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
        (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (skip_connection): Conv1d(28, 128, kernel_size=(1,), stride=(1,))
      )
      (2): ResidualBlock(
        (conv1): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (bn1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv1d(256, 256, kernel_size=(3,), stride=(1,), padding=(1,))
        (bn2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (skip_connection): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      )
 

# Training Config
Initialize Loss Criterion, Optimizer, CTC Beam Decoder, Scheduler, Scaler (Mixed-Precision), etc.

In [21]:
#TODO


criterion = torch.nn.CTCLoss(blank=0, reduction='mean', zero_infinity=False) # Define CTC loss as the criterion. How would the losses be reduced?
# CTC Loss: https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html
# Refer to the handout for hints

optimizer =  torch.optim.AdamW(model.parameters(), lr= config['lr']) # What goes in here?

# Declare the decoder. Use the CTC Beam Decoder to decode phonemes
# CTC Beam Decoder Doc: https://github.com/parlance/ctcdecode
decoder = CTCBeamDecoder(LABELS, beam_width =config['beam_width'], log_probs_input = True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.6,patience=1)

# Mixed Precision, if you need it
scaler = torch.cuda.amp.GradScaler()

# Decode Prediction

In [ ]:
def decode_prediction(output, output_lens, decoder, PHONEME_MAP= LABELS):
    
    # TODO: look at docs for CTC.decoder and find out what is returned here. Check the shape of output and expected shape in decode.
    beam_results, beam_scores, timesteps, out_seq_len = decoder.decode(output,seq_lens=output_lens) #lengths - list of lengths

    pred_strings  = []
    # print(beam_results.shape)
    # print(beam_results)
    for i in range(output_lens.shape[0]):
        #TODO: Create the prediction from the output of decoder.decode. Don't forget to map it using PHONEMES_MAP.
        pred_strings.append(''.join([PHONEME_MAP[n] for n in beam_results[i][0][:out_seq_len[i][0]]]))
    # print(pred_strings)
    
    return pred_strings

def calculate_levenshtein(output, label, output_lens, label_lens, decoder, PHONEME_MAP= LABELS): # y - sequence of integers
    
    dist            = 0
    batch_size      = label.shape[0]

    pred_strings    = decode_prediction(output, output_lens, decoder, PHONEME_MAP)
    # print(batch_size)
    for i in range(batch_size):
        # TODO: Get predicted string and label string for each element in the batch
        pred_string = pred_strings[i]#TODO
        # print('pred',pred_string)
        label_string = ''.join([PHONEME_MAP[n] for n in label[i][:label_lens[i]]]) #TODO
        # print('label',label_string)
        dist += Levenshtein.distance(pred_string, label_string)

    dist /= batch_size # TODO: Uncomment this, but think about why we are doing this
    
    return dist

# Test Implementation

In [23]:
model.eval()
for i, data in enumerate(val_loader, 0):
    x, y, lx, ly = data
    x, y = x.to(device), y.to(device)
    h, lh = model(x, lx)
    print(h.shape)
    h = torch.permute(h, (1, 0, 2))
    print(h.shape, y.shape)
    loss = criterion(h, y, lh, ly)
    print(loss)
    print(calculate_levenshtein(h, y, lx, ly, decoder, LABELS))
    break

torch.Size([64, 734, 41])
torch.Size([734, 64, 41]) torch.Size([64, 265])
tensor(7.6873, device='cuda:0', grad_fn=<MeanBackward0>)
62.28125


# WandB

You will need to fetch your api key from wandb.ai

In [24]:
import wandb
wandb.login(key="d55e83a86cb97a0c3653945cccd0eed009d488a1")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [25]:
run = wandb.init(
    name = "early-submission", ## Wandb creates random run names if you skip this field
    reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # run_id = ### Insert specific run id here if you want to resume a previous run
    # resume = "must" ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw3p2-ablations", ### Project should be created in your wandb account
    config = config ### Wandb Config for your run
)

wandb: Currently logged in as: shunl (shunl-carnegie-mellon-university). Use `wandb login --relogin` to force relogin


# Train Functions

In [26]:
from tqdm import tqdm

def train_model(model, train_loader, criterion, optimizer):

    model.train()
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    total_loss = 0

    for i, data in enumerate(train_loader):
        optimizer.zero_grad()

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.cuda.amp.autocast():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += loss.item()

        batch_bar.set_postfix(
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            lr="{:.06f}".format(float(optimizer.param_groups[0]['lr'])))

        batch_bar.update() # Update tqdm bar

        # Another couple things you need for FP16.
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close() # You need this to close the tqdm bar

    return total_loss / len(train_loader)


def validate_model(model, val_loader, decoder, phoneme_map= LABELS):

    model.eval()
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    total_loss = 0
    vdist = 0

    for i, data in enumerate(val_loader):

        x, y, lx, ly = data
        x, y = x.to(device), y.to(device)

        with torch.inference_mode():
            h, lh = model(x, lx)
            h = torch.permute(h, (1, 0, 2))
            loss = criterion(h, y, lh, ly)

        total_loss += float(loss)
        vdist += calculate_levenshtein(torch.permute(h, (1, 0, 2)), y, lh, ly, decoder, phoneme_map)

        batch_bar.set_postfix(loss="{:.04f}".format(float(total_loss / (i + 1))), dist="{:.04f}".format(float(vdist / (i + 1))))

        batch_bar.update()

        del x, y, lx, ly, h, lh, loss
        torch.cuda.empty_cache()

    batch_bar.close()
    total_loss = total_loss/len(val_loader)
    val_dist = vdist/len(val_loader)
    return total_loss, val_dist

## Training Setup

In [27]:
def save_model(model, optimizer, scheduler, metric, epoch,path):
    torch.save(
        {'model_state_dict'         : model.state_dict(),
         'optimizer_state_dict'     : optimizer.state_dict(),
         'scheduler_state_dict'     : scheduler.state_dict(),
         metric[0]                  : metric[1],
         'epoch'                    : epoch},
        path
    )

def load_model(path, model, optimizer= None, scheduler= None):

    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])

    if optimizer != None:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    if scheduler != None:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

    return model

In [28]:
# This is for checkpointing, if you're doing it over multiple sessions

last_epoch_completed = 0
start = last_epoch_completed
end = config["epochs"]
best_lev_dist = float("inf") # if you're restarting from some checkpoint, use what you saw there.

In [29]:
torch.cuda.empty_cache()
gc.collect()

#TODO: Please complete the training loop

for epoch in range(0, config['epochs']):

    print("\nEpoch: {}/{}".format(epoch+1, config['epochs']))

    curr_lr = float(optimizer.param_groups[0]['lr'])

    train_loss              = train_model(model, train_loader, criterion, optimizer)
    valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)
    scheduler.step(valid_dist)
    


    print("\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_loss, curr_lr))
    print("\tVal Dist {:.04f}%\t Val Loss {:.04f}".format(valid_dist, valid_loss))


    wandb.log({
        'train_loss': train_loss,
        'valid_dist': valid_dist,
        'valid_loss': valid_loss,
        'lr'        : curr_lr
    })
    '''
    save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch)
    wandb.save(epoch_model_path)
    print("Saved epoch model")
    '''

    if valid_dist <= best_lev_dist:
        best_lev_dist = valid_dist
        save_model(model, optimizer, scheduler, ['valid_dist', valid_dist], epoch, '/kaggle/working/best_ret.pth')
        wandb.save('/kaggle/working/best_ret.pth')
        print("Saved best model")
      # You may find it interesting to exlplore Wandb Artifcats to version your models
run.finish()


Epoch: 1/90


	Train Loss 1.9622	 Learning Rate 0.0020000
	Val Dist 22.5260%	 Val Loss 0.9644


wandb: WARNING Saving files without folders. If you want to preserve subdirectories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")


Saved best model

Epoch: 2/90


	Train Loss 1.0011	 Learning Rate 0.0020000
	Val Dist 16.1877%	 Val Loss 0.6751
Saved best model

Epoch: 3/90


...

	Train Loss 0.2938	 Learning Rate 0.0000560
	Val Dist 6.3311%	 Val Loss 0.2660
Saved best model

Epoch: 52/90


	Train Loss 0.2952	 Learning Rate 0.0000560
	Val Dist 6.3377%	 Val Loss 0.2654

Epoch: 53/90


	Train Loss 0.2916	 Learning Rate 0.0000560
	Val Dist 6.3280%	 Val Loss 0.2654
Saved best model

Epoch: 54/90


Train:  74%|███████▍  | 329/446 [06:12<02:09,  1.11s/it, loss=0.2984, lr=0.000056]

KeyboardInterrupt: 

In [ ]:
decoder = CTCBeamDecoder(LABELS, beam_width =30, log_probs_input = True)
valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)
print(valid_dist)

In [ ]:
decoder = CTCBeamDecoder(LABELS, beam_width =1, log_probs_input = True)
valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)
print(valid_dist)

In [ ]:
decoder = CTCBeamDecoder(LABELS, beam_width =40, log_probs_input = True)
valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)
print(valid_dist)

In [ ]:
decoder = CTCBeamDecoder(LABELS, beam_width =3, log_probs_input = True)
valid_loss, valid_dist  = validate_model(model, val_loader, decoder, phoneme_map= LABELS)
print(valid_dist)

In [ ]:
model=load_model('/kaggle/working/best_ret.pth', model)

# Generate Predictions and Submit to Kaggle

In [ ]:
#TODO: Make predictions

# Follow the steps below:
# 1. Create a new object for CTCBeamDecoder with larger (why?) number of beams
# 2. Get prediction string by decoding the results of the beam decoder

TEST_BEAM_WIDTH = 200

test_decoder = CTCBeamDecoder(LABELS, beam_width =TEST_BEAM_WIDTH, log_probs_input = True)
results = []

model.eval()
print("Testing")
for data in tqdm(test_loader):

    x, lx   = data
    x       = x.to(device)

    with torch.no_grad():
        h, lh = model(x, lx)

    prediction_string= decode_prediction(h,lh, test_decoder, LABELS)
    
    results.extend(prediction_string)
    
    del x, lx, h, lh
    torch.cuda.empty_cache()

In [ ]:
print(results[0])

In [ ]:
data_dir = f"/kaggle/input/example/random_submission.csv"
df = pd.read_csv(data_dir)
df.label = results
df.to_csv('submission.csv', index = False)

In [ ]:
!kaggle competitions submit -c hw3p2-785-f24 -f submission.csv -m "I made it!"
